In [1]:
############ LOAD PACKAGES
import numpy as np
import pandas as pd
import napari
import tifffile as tf
import matplotlib.pyplot as plt

# import custum colormaps
from colormaps import *

# import custum functions
from STP_plotting import *
from STP_processing import *

# auto refreshes imported functions if source code changes:
%load_ext autoreload
%autoreload 2

In [2]:
# set-up vars
home_dir = "/Volumes/Data/"
# home_dir = "/mnt/labNAS/"

in_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/input_tifs/"
out_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/output_figs/"

metadata = pd.read_csv("stp_metadata.csv")

In [10]:
# import example singing mouse
# import hand-straight brain
steg_to_st_tif = in_path+"norm_subtracted/STeg_220429_aligned/OMC_STeg_220208_b0_aligned_to_STeg_220429_NO_subtracted.tif"

# import 230429 STeg brain- seems to be brighter than others???
steg_hs_asr = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_STeg_220429_b2_hand_straightened_asr.tif"

# # import example lab mouse
# mmus_tif = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_MMus_220303_b1_hand_straightened_asr.tif"


# import example lab mouse aligned to sm
mmus_to_st_tif = in_path+"norm_subtracted/STeg_220429_aligned/OMC_MMus_220119_b0_aligned_to_STeg_220429_NO_subtracted.tif"


# load tifs
steg_to_st = tf.imread(steg_to_st_tif)
# mmus = tf.imread(mmus_tif)
steg = tf.imread(steg_hs_asr)
mmus_to_st_b0 = tf.imread(mmus_to_st_tif)


In [4]:

# import boundaries
# import aligned to hand-straight
boundaries_path = home_dir+"Emily/STP_for_MAPseq/3_brainreg_output/OMC_STeg_220429_b2_hand_straightened_asr_aligned_10um/boundaries_RESIZED.tif"
boundaries = tf.imread(boundaries_path)




In [11]:
viewer = napari.Viewer()

# # add example image
# viewer.add_image(
#     steg_to_st,
#     scale=[50,20,20],
#     blending="additive",
#     colormap="bop orange"
# )

# add example image
viewer.add_image(
    steg,
    scale=[50,20,20],
    blending="additive",
    colormap="bop orange"
)

# # add example image
# viewer.add_image(
#     mmus,
#     scale=[2.5,1,1],
#     blending="additive"
# )

# add example image
viewer.add_image(
    mmus_to_st_b0,
    scale=[50,20,20],
    blending="additive",
    colormap="bop blue"
)

# # add example image
# viewer.add_image(
#     boundaries,
#     scale=[50,20,20],
#     blending="additive"
# )


<Image layer 'mmus_to_st_b0' at 0x17a1ccb80>

In [31]:
# add example image
viewer.add_image(
    boundaries,
    scale=[50,20,20],
    blending="additive"
)

<Image layer 'boundaries [1]' at 0x180309d80>

In [12]:
# add scale bar
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"
# viewer.scale_bar.

In [21]:
# manipulate in napari until get orientation you want...
# take screenshot, specify size for highres
file_name = "st230429_mm220119_align220429_z97_st_mm.png"
size = steg_to_st.shape[1:]
viewer.screenshot(out_path+file_name, canvas_only=True, size=(size))
print("done")

done


: 